In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, IFrame
from ipywidgets import interact,fixed
import pandas as pd
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import FancyArrowPatch,Rectangle
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import proj3d

plt.rcParams["figure.figsize"] = [10, 10]

from numpy.linalg import norm
from numpy import cos,sin,tan,arctan,exp,log,pi,sqrt,arccos,linspace,array,arange,meshgrid,abs

from scipy.integrate import quad,dblquad,tplquad

## One-minute Review

A homework question asked about an integral of the form 

$$ \int_0^1\int_0^z\int_z^1 f\,dx\,dy\,dz $$

In [2]:
@interact(angle=(-0,90,6),a=(0.,1.,.05))
def _(angle=-24,vangle=(0,90,6),a=0):
    fig = plt.figure(figsize=(10,10))
    ax= fig.add_subplot(111,projection='3d')
    u = np.linspace(0,1,80)
    v = np.linspace(0,1,80)
    U,V = np.meshgrid(u,v)
    X = U
    Y = V
    Z = np.ones_like(U)
#     ax.plot_wireframe(X,Y,Z,rcount=20,ccount=20)
#     ax.plot_surface(X,Y,0*X,rcount=20,ccount=20,color='gray',alpha=.4)
    ax.view_init(vangle,angle)
    
    ax.plot_wireframe(U,V*(U),U,rcount=20,ccount=20,alpha=a)
    ax.plot_wireframe(U*0+1,V*(U),U,rcount=20,ccount=20,alpha=a)
    ax.plot_wireframe(U+V*(1-U),U,U,rcount=20,ccount=20,alpha=a)
    ax.plot_surface(U,0*X,V*(U),rcount=20,ccount=20,color='gray',alpha=.4)
    ax.plot_surface(U*0,U,U+V*(1-U),rcount=20,ccount=20,color='gray',alpha=.4)

    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=0, description='angle', max=90, step=6), IntSlider(value=42, description…

# Cylindrical/Spherical Coordinates

### Quick Exercise

We know the equations $x=1$, $y=1$, and $z=1$ represent planes perpendicular to the respective access. Identify each of the following sets in the new coordinates.

  1. $r=1$
  2. $\theta = 1$
  3. $\rho = 1$
  4. $\phi = 1$

In [3]:
@interact
def _(angle=(-90,90,6),coord=['r','theta','rho','phi']):
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(30,angle)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)
    if coord == 'r':
        ax.plot_surface(cos(2*pi*v),sin(2*pi*v),2*u-1,alpha=.5)
    elif coord == 'theta':
        ax.plot_surface(sqrt(2)*v*cos(1),sqrt(2)*v*sin(1),2*u-1,alpha=.5)
    elif coord == 'rho':
        ax.plot_surface(sin(pi*u)*cos(2*pi*v),sin(pi*u)*sin(2*pi*v),cos(pi*u),alpha=.5)
    elif coord == 'phi':
        ax.plot_surface(sin(1)*u*cos(2*pi*v),sin(1)*u*sin(2*pi*v),cos(1)*u,alpha=.5)
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-1,1],'k',lw=3)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=0, description='angle', max=90, min=-90, step=6), Dropdown(description='…

## Exercise

Each row of the table below represents a point in $\mathbb{R}^3$. Fill in the missing values.

<table>
    <hr>
    <td> $x$ </td>    <td> $y$ </td>    <td> $z$ </td>
    <td> $r$ </td>    <td> $\theta$ </td>    <td> $\rho$ </td>    <td> $\phi$ </td>
    </hr>
    <tr>
    <td> $2$ </td>    <td> $2$ </td>    <td> $2$ </td>
    <td>  </td>    <td>  </td>    <td>  </td>    <td>  </td>
    </tr>
    <tr>
    <td>   </td>    <td>   </td>    <td> $2$ </td>
    <td> $3/2$ </td>    <td> $\pi$ </td>    <td>  </td>    <td>  </td>
    </tr>
    <tr>
    <td>  </td>    <td>  </td>    <td>  </td>
    <td>  </td>    <td> $\pi/6$ </td>    <td> $\sqrt{2}$ </td>    <td> $3\pi/4$</td>
    </tr>
    <tr>
    <td> $5$ </td>    <td> $1$ </td>    <td>  </td>
    <td>  </td>    <td>  </td>    <td>  </td>    <td> $2\pi/3$ </td>
    </tr>
</table>

## Latitude/Longitude

The surface of the earth is [roughly](https://en.wikipedia.org/wiki/Figure_of_the_Earth#/media/File:Earth_oblateness_to_scale.svg) a sphere. 

Longitude measures degrees east (+) or west (-) from the Greenwich meridiean (a line from the North Pole to the South Pole through Greenwich, England). Latitude measures degrees north (+) or south (-) of the equator. 

#### Quick questions

How would you relate these to our definitions of $\phi$ and $\theta$?

Using this, what are the spherical coordinates of Morningside Heights?

Where is the point on earth where $\phi = \frac\pi4 = -\theta$?

## Example

Find, in terms of constant $a>0$, the centroid of the solid region

$$ 0 \leq z \leq a^2 - x^2 - y^2.$$

In [4]:
@interact
def _(angle=(0,90,6)):
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(angle,30)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)

    ax.plot_surface(sqrt(2)*u*cos(2*pi*v),sqrt(2)*u*sin(2*pi*v),2*(1-u**2),alpha=.5)
    ax.plot_wireframe(sqrt(2)*u*cos(2*pi*v),sqrt(2)*u*sin(2*pi*v),2*(1-u**2),alpha=1,color='k',ccount=10,rcount=10)
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[0,2],'k',lw=3)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=42, description='angle', max=90, step=6), Output()), _dom_classes=('widg…

## Example

Find the mass of a right cone with height $h$ and radius (at base) $R$. and uniform density $\rho$.

In [5]:
@interact
def _(angle=(0,90,6)):
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(angle,30)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)

    ax.plot_surface(sin(1)*u*cos(2*pi*v),sin(1)*u*sin(2*pi*v),2-2*u,alpha=.5)
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[0,2],'k',lw=3)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=42, description='angle', max=90, step=6), Output()), _dom_classes=('widg…

<br>
<div style="background:#cfcfee">**Caution!** When doing these sorts of applications, do not confuse the density $\rho$ with the polar coordinate $\rho$. Choose variables wisely.</div>

Find the moment of inertia relative to its central axis. 

## Example

Find the centroid of a solid quarter-sphere of radius $R$ and uniform density.

In [6]:
@interact
def _(angle=(-45,135,6),vangle=(0,90,6)):
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(vangle,angle)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)

    ax.plot_wireframe(sin(pi*u)*cos(pi/2*v),sin(pi*u)*sin(pi/2*v),cos(pi*u),alpha=.8,rcount=20)
    ax.plot_surface(v*sin(pi*u)*cos(pi/2),v*sin(pi*u)*sin(pi/2),v*cos(pi*u),alpha=.3,color='g')
    ax.plot_surface(v*sin(pi*u)*cos(0),v*sin(pi*u)*sin(0),v*cos(pi*u),alpha=.3,color='g')
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-1,1],'k',lw=3)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=45, description='angle', max=135, min=-45, step=6), IntSlider(value=42, …

## Example

Find the moment of inertia of a solid sphere of radius $R$ relative to a central axis.

The **radius of gyration** of a body is the distance from the axis of a point-mass with the same mass and moment of inertia. Find it for a sphere.

In [7]:
@interact
def _(angle=(0,90,6)):
    tilt = 23.5*pi/180
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(angle,30)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)

    ax.plot_wireframe(sin(pi*u)*cos(2*pi*v),sin(pi*u)*sin(2*pi*v),cos(pi*u),alpha=1,rcount=12,ccount = 20)
    ax.plot_surface(sin(pi*u)*cos(2*pi*v),sin(pi*u)*sin(2*pi*v),cos(pi*u),alpha=.2,color='g')
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-1,1],'k',lw=3)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=42, description='angle', max=90, step=6), Output()), _dom_classes=('widg…

## Example

The Earth has a mass of roughly $6\times10^{24}$ kg. Assuming it's uniformly dense, what is the mass of the portion above the arctic circle ($66.5^\circ$N)?

In [8]:
@interact
def _(angle=(0,90,6)):
    tilt = 23.5*pi/180
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(angle,30)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)

    ax.plot_wireframe(sin(tilt*u)*cos(2*pi*v),sin(tilt*u)*sin(2*pi*v),cos(tilt*u),alpha=1,rcount=10,ccount = 5)
    ax.plot_wireframe(sin((1-u)*tilt + pi*u)*cos(2*pi*v),sin((1-u)*tilt + pi*u)*sin(2*pi*v),cos((1-u)*tilt + pi*u),rcount=10,ccount = np.ceil(5*(pi-tilt)/tilt),alpha=.4)
    ax.plot_surface(sin(tilt*u)*cos(2*pi*v),sin(tilt*u)*sin(2*pi*v),0*u + cos(tilt),alpha=.3,color='g')
    ax.plot([-1,1],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1,1],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-1,1],'k',lw=3)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=42, description='angle', max=90, step=6), Output()), _dom_classes=('widg…

In [9]:
alpha = 23.5*pi/180

In [10]:
R = 6371

In [11]:
tplquad(lambda rho, ph, th: rho**2*sin(ph),0,2*pi,0,alpha,lambda th,ph: R*cos(alpha)/cos(ph),R)

(5434055746.573576, 6.033013806942629e-05)

## Exercise

Set up a triple integral to find the volume of a torus with radii $a < b$.

In [12]:
@interact
def _(angle=(0,90,6)):
    tilt = 23.5*pi/180
    fig = plt.figure(figsize=(10,10))
    ax=fig.add_subplot(111,projection='3d')
    ax.view_init(angle,30)
    u = v = np.linspace(0,1,100)
    u,v = np.meshgrid(u,v)

    ax.plot_wireframe((1+1/2*sin(2*pi*u))*cos(2*pi*v),(1+1/2*sin(2*pi*u))*sin(2*pi*v),(1/2*cos(2*pi*u)),alpha=1,rcount=10,ccount = 10)
    ax.plot_surface((1+1/2*sin(2*pi*u))*cos(2*pi*v),(1+1/2*sin(2*pi*u))*sin(2*pi*v),(1/2*cos(2*pi*u)),alpha=.3,color='g')
    ax.plot([-1.5,1.5],[0,0],[0,0],'k',lw=3)
    ax.plot([0,0],[-1.5,1.5],[0,0],'k',lw=3)
    ax.plot([0,0],[0,0],[-1.5,1.5],'k',lw=3)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    

interactive(children=(IntSlider(value=42, description='angle', max=90, step=6), Output()), _dom_classes=('widg…